In [30]:
import cv2
import numpy as np
import itertools
from sklearn import linear_model
import matplotlib.pyplot as plt
from rich import print
from numba import jit

# Load Image

In [12]:
image = cv2.imread('/home/haohang/CS-558/hw2/result/keypoint.png', 0)

print(image.shape)
print(image.dtype)
print(np.unique(image))

(407, 548)

float32

[  0. 255.]

In [20]:
y, x = np.where(image > 0)
y, x = x.astype(np.float32), y.astype(np.float32)

points = np.stack([x, y], axis=1)

print(points[:5])
print(points.shape)

[[  1. 256.]
 [  1. 258.]
 [  1. 260.]
 [  1. 262.]
 [  1. 264.]]

(778, 2)

605284

## Sample Points

In [56]:
def random_n_points_pair(points, n):
    idx = np.random.choice(points.shape[0], n, replace=False)
    return np.array(list(itertools.combinations(points[idx], 2)))[:n]

In [58]:
sampled_points = random_n_points_pair(points, 10)
print(sampled_points)

[[[252.  82.]
  [210. 223.]]

 [[252.  82.]
  [ 80. 231.]]

 [[252.  82.]
  [169. 441.]]

 [[252.  82.]
  [ 76. 245.]]

 [[252.  82.]
  [ 79. 289.]]

 [[252.  82.]
  [143. 243.]]

 [[252.  82.]
  [192.  31.]]

 [[252.  82.]
  [111. 320.]]

 [[252.  82.]
  [ 61. 323.]]

 [[210. 223.]
  [ 80. 231.]]]

# Compute Line

In [82]:
def fit_line1(p1, p2):
    x1, y1 = p1
    x2, y2 = p2

    # Calculating the coefficients
    a = y2 - y1
    b = x1 - x2
    c = x2 * y1 - x1 * y2

    # Normalize the coefficients so that a^2 + b^2 = 1
    norm = (a**2 + b**2)**0.5
    a, b, c = a / norm, b / norm, c / norm

    return a, b, c

In [83]:
p1 = np.array([1, 2])
p2 = np.array([3, 4])
a, b, c = fit_line1(p1, p2)
print(f"The line coefficients are: a = {a}, b = {b}, c = {c}")

The line coefficients are: a = 0.7071067811865475, b = -0.7071067811865475, c = 0.7071067811865475

In [88]:
def fit_line(p1, p2):
    m = np.vstack([p1, p2])
    m = np.hstack([m, np.ones((2, 1))])
    eigen_values, eigen_vectors = np.linalg.eig(m.T @ m)
    index = np.argmin(eigen_values)
    a, b, c = eigen_vectors[:, index]
    norm = (a**2 + b**2)**0.5
    a, b, c = a / norm, b / norm, c / norm
    return a, b, c

In [89]:
p1 = np.array([1, 2])
p2 = np.array([3, 4])
a, b, c = fit_line(p1, p2)
print(f"The line coefficients are: a = {a}, b = {b}, c = {c}")

The line coefficients are: a = 0.7071067811865475, b = -0.7071067811865477, c = 0.7071067811865483

# Distance From A Line

In [91]:
def distance_to_line(p, a, b, c):
    x, y = p
    return abs(a * x + b * y + c)

In [98]:
distance_to_line(np.array([3, 4]), 1, -1, 1)

0